In [1]:
from transformers import AutoModel, AutoTokenizer
from huggingface_hub import HfApi, Repository, HfFolder
from requests.exceptions import HTTPError

In [13]:
def push_model_to_huggingface(model_path, repo_name, huggingface_token, commit_message="Initial commit"):
    # Hugging Face API ve Giriş
    HfFolder.save_token(huggingface_token)
    api = HfApi()

    # Tokenizer ve Model Yükleme
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path)

    # Repository Yaratma veya Klonlama
    repo_url = api.create_repo(repo_name, exist_ok=True)
    repo = Repository(local_dir=repo_name, clone_from=repo_url)

    # Uzak değişiklikleri çekme
    try:
        repo.git_pull()
    except Exception as e:
        print(f"Error pulling from remote: {e}")

    # Model ve Tokenizer'ı Kaydetme
    model.save_pretrained(repo_name)
    tokenizer.save_pretrained(repo_name)

    # Dosyaları Commit ve Push Etme
    repo.git_add()
    try:
        repo.git_commit(commit_message)
    except EnvironmentError as e:
        if "nothing to commit" in str(e):
            print("No changes to commit.")
        else:
            raise e

    try:
        repo.git_push()
    except EnvironmentError as e:
        print(f"Error pushing to remote: {e}")
        print("Trying to pull changes and push again.")
        repo.git_pull()
        repo.git_push()

    return repo_url

In [3]:
base_path = "/content/drive/My Drive/yl_tez"

In [4]:
import os
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Yerelde Çalışıyor.")
    base_path = "."

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
model_adi = "BestModelParameters"
model_yolu = os.path.join(base_path, model_adi)
repo_name = "kayrab/ytu_doktor_gpt2-mediuma"
commit_message = "eyvallah"
huggingface_token = ""

In [ ]:
# Kullanım
repo_url = push_model_to_huggingface(model_yolu, repo_name, huggingface_token, commit_message)
print(f"Model başarıyla güncellendi: {repo_url}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/kayrab/ytu_doktor_gpt2-mediuma into local empty directory.
